# Polymarket User PnL Calculation (with Market Resolution)

This notebook calculates the Profit and Loss (PnL) for users based on trade history and market resolution status.

**Methodology: Cash Flow**
$$ \text{Total PnL} = \text{Net Cash Flow} + \text{Position Value} $$

- **Net Cash Flow**: Total money received from sales minus total money spent on buys.
- **Position Value**: 
    - If market is **OPEN**: `Shares Held * Current Price` (Last trade price).
    - If market is **RESOLVED**: `Shares Held * (1.0 if winner else 0.0)`.


In [2]:
import pandas as pd
import numpy as np
import json

# 1. Load Data
# trades_path = '../docs/sample_trades.csv'
# markets_path = '../docs/sample_market.csv'

trades_path = 'data/trades.csv'
markets_path = 'data/markets.csv'

# Handle potential encoding issues
df_trades = pd.read_csv(trades_path, encoding='utf-8', encoding_errors='replace')
df_markets = pd.read_csv(markets_path, encoding='utf-8', encoding_errors='replace')

print(f"Loaded {len(df_trades)} trades and {len(df_markets)} markets.")

Loaded 2140596 trades and 12331 markets.


## 2. Process Market Resolution Data

Extract resolution status and winning outcome prices from the JSON `data` column.

In [3]:
market_resolution_map = {}

for idx, row in df_markets.iterrows():
    try:
        data = json.loads(row['data'])
        slug = row['slug']
        is_closed = data.get('closed', False)
        
        if is_closed:
            # Polymarket 'outcomes' and 'payouts' are usually parallel arrays
            outcomes = json.loads(data.get('outcomes', '[]'))
            payouts = data.get('resolution', {}).get('payouts', [])
            
            if outcomes and payouts:
                # Map outcome name -> payout price (1.0 or 0.0)
                res_info = {outcomes[i]: float(payouts[i]) for i in range(len(outcomes))}
                market_resolution_map[slug] = res_info
    except Exception as e:
        continue

print(f"Processed {len(market_resolution_map)} resolved markets.")

Processed 539 resolved markets.


## 3. Preprocess Trades

Calculate individual trade cash flow relative to the user.

In [4]:
def calculate_flow(row):
    if row['side'] == 'BUY':
        return -row['usdc_volume'], row['shares']
    else: # SELL
        return row['usdc_volume'], -row['shares']

df_trades[['cash_flow', 'share_change']] = df_trades.apply(
    lambda row: pd.Series(calculate_flow(row)), axis=1
)

## 4. Aggregation & Valuation

Group by user/market/outcome and apply valuation based on market state.

In [6]:
# Aggregating positions
pnl_df = df_trades.groupby(['maker', 'market_slug', 'outcome']).agg(
    net_cash_flow=('cash_flow', 'sum'),
    net_shares=('share_change', 'sum')
).reset_index()

# Get latest market price as fallback for open markets
df_trades['timestamp'] = pd.to_datetime(df_trades['timestamp'], format='mixed')
latest_prices = df_trades.sort_values('timestamp').groupby(['market_slug', 'outcome']).tail(1)
latest_prices_map = latest_prices.set_index(['market_slug', 'outcome'])['price'].to_dict()

def value_position(row):
    market = row['market_slug']
    outcome = row['outcome']
    shares = row['net_shares']
    
    # If market is resolved, use resolution price
    if market in market_resolution_map and outcome in market_resolution_map[market]:
        return shares * market_resolution_map[market][outcome]
    
    # Fallback: Current market price
    curr_price = latest_prices_map.get((market, outcome), 0)
    return shares * curr_price

pnl_df['position_value'] = pnl_df.apply(value_position, axis=1)
pnl_df['total_pnl'] = pnl_df['net_cash_flow'] + pnl_df['position_value']

pnl_df.sort_values('total_pnl', ascending=False).head(10)

,maker,market_slug,outcome,net_cash_flow,net_shares,position_value,total_pnl
56270,0x16b29C50f2439faf627209B2AC0c7BbDDaA8a881,nfl-la-sea-2025-12-18,Rams,-581594.950300,1.269311e+06,837745.187400,256150.237100
136507,0x3657862E57070b82A289b5887EC943A7C2166b14,nhl-phi-mon-2025-12-16,Flyers,218954.498751,-3.637892e+02,-363.425379,218591.073372
624715,0xf98DEc26318D00d7863746D6Aa2347691EE37ADb,nba-nyk-ind-2025-12-18,Knicks,-166690.587100,2.626523e+05,262389.627720,95699.040620
413935,0xAfbAcAEEda63f31202759EFF7f8126E49ADfe61B,nba-sas-nyk-2025-12-16,Knicks,-116414.396000,2.078829e+05,207674.967150,91260.571150
56251,0x16b29C50f2439faf627209B2AC0c7BbDDaA8a881,nba-sas-nyk-2025-12-16-spread-home-2pt5,Knicks,-78273.145000,1.588929e+05,158734.027080,80460.882080
61290,0x18d12d50Db693Ea3258EDaA721c8fFa6EB7BDbCb,nba-nyk-ind-2025-12-18,Knicks,74913.610904,1.522000e-03,0.001520,74913.612424
624711,0xf98DEc26318D00d7863746D6Aa2347691EE37ADb,nba-atl-cha-2025-12-18,Hornets,-39598.008800,1.111239e+05,111012.746130,71414.737330
274008,0x6f2628a8Ac6E3F7bD857657d5316c33822CED136,bitcoin-above-88k-on-december-18,No,-21385.827292,9.085859e+04,90858.593857,69472.766565
357540,0x91654fD592Ea5339Fc0B1b2f2b30bFfFA5e75b98,nba-lac-okc-2025-12-18-spread-home-17pt5,Thunder,-172824.925500,3.398119e+05,237868.309000,65043.383500
132635,0x3450A5a6C9F7b35caA976D2D064b51dAB3Bd6793,nba-nyk-ind-2025-12-18,Knicks,-17489.268697,7.638947e+04,76313.084746,58823.816049


## 5. User Leaderboard

In [9]:
user_leaderboard = pnl_df.groupby('maker').agg({
    'total_pnl': 'sum',
    'net_cash_flow': 'sum',
    'position_value': 'sum'
}).reset_index()

user_leaderboard.sort_values('total_pnl', ascending=False, inplace=True)
user_leaderboard.head(100)

user_leaderboard.to_csv('data/user_leaderboard.csv', index=False)